# To create your scene, scroll down to Create Your scene
場面を実際に作るには、「あなたの場面をつくる」までスクロールダウンしてください。

In [ ]:
#!pip install --upgrade openai
#!pip install langchain  --ignore-installed PyYAML
#!pip install tiktoken
#!pip install python-dotenv

In [ ]:
import openai
import json
import tiktoken
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage
from langchain.schema import HumanMessage
from langchain.schema import SystemMessage
from langchain import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
import datetime
from dotenv import load_dotenv
import os

In [ ]:
# OPENAI_API_KEY = "Replace here with your API key"

# if you saved your API key in .env file, you can use the following code.
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
def is_valid_json(input_string):
    if input_string == None or input_string == "":
      return False

    try:
        json.loads(input_string)
        return True
    except:
        return False

def decode_nested_json(data):
    if isinstance(data, dict):
        return {k: decode_nested_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [decode_nested_json(v) for v in data]
    elif isinstance(data, str):
        try:
            return decode_nested_json(json.loads(data))
        except json.JSONDecodeError:
            return data
    else:
        return data


def save_dict_as_json(dict_data, file_name_without_extension):
    text_data = json.dumps(dict_data, indent = 4)
    with open(f'result/{file_name_without_extension}.txt', 'w', encoding='utf-8') as f:
            f.write(text_data)

In [ ]:
def gpt_response_from_lists(system_content_list = [], user_content_list = [], assistant_content_list = [], max_tokens = None, model_name = "gpt-4"):
    print(f"gpt_response_from_lists start. TIME: {datetime.datetime.now()}")

    # check if the model name is valid
    model_list = openai.Model.list()
    model_names = [model["id"] for model in model_list["data"]]
    if not model_name in model_names:
      print(f"model_name {model_name} is unavailable. I'll use gpt-3.5-turbo instead.")
      model_name = "gpt-3.5-turbo"

    messages = [] 
    concatenated_messages = "" # this text is used to estimate the total tokens

    for system_content in system_content_list:
      messages.append(SystemMessage(content = system_content))
      concatenated_messages += system_content

    list_length = max(len(assistant_content_list), len(user_content_list))

    for i in range(0, list_length):

      if i < len(user_content_list):
        messages.append(HumanMessage(content = user_content_list[i]))
        concatenated_messages += user_content_list[i]

      if i < len(assistant_content_list):
        messages.append(AIMessage(content = assistant_content_list[i]))
        concatenated_messages += assistant_content_list[i]

    encoding = tiktoken.encoding_for_model(model_name)
    encoded_text = encoding.encode(concatenated_messages)
    len_encoded_text = len(encoded_text)

    tokens_per_text = 3
    tokens_per_name = 1

    name_count = len(system_content_list) + len(assistant_content_list) + len(user_content_list)
    total_tokens = name_count * tokens_per_name + len_encoded_text * tokens_per_text

    print(f"estimated total tokens: {total_tokens}, model name: {model_name}")

    chat = ChatOpenAI(model_name=model_name, openai_api_key= OPENAI_API_KEY)

    if max_tokens != None:
      chat = ChatOpenAI(model_name=model_name, openai_api_key= OPENAI_API_KEY, max_tokens=max_tokens)

    try:
      response = chat(messages)
    except Exception as e:
      try: 
        print(f"error in gpt_response_from_lists: {e}. I'll retry with gpt-3.5-turbo-16k")
        model_name = "gpt-3.5-turbo-16k"
        chat = ChatOpenAI(model_name=model_name, openai_api_key= OPENAI_API_KEY) if max_tokens == None else ChatOpenAI(model_name=model_name, openai_api_key= OPENAI_API_KEY, max_tokens=max_tokens)
        response = chat(messages)
      except Exception as e2:
        print(f"error in gpt_response_from_lists: {e}. I'll retry with gpt-3.5-turbo-16k, without assistant_content_list")
        messages = [] 

        for system_content in system_content_list:
          messages.append(SystemMessage(content = system_content))

        for user_content in user_content_list:
          messages.append(HumanMessage(content = user_content))
          chat = ChatOpenAI(model_name=model_name, openai_api_key= OPENAI_API_KEY) 
          response = chat(messages)


    print(f"response= {response}")
    print(f"gpt_response_from_lists done TIME: {datetime.datetime.now()}")

    return response

In [ ]:
class Action_Object:
    def __init__(self, action = "", who_took_action = "Anonymous", consequence = "Undetermined", utterance="",  score = 0.0):
        self.action = action
        self.who_took_action = who_took_action
        self.consequence = consequence
        self.score = score
        self.utterance = utterance
    
    def __str__(self):
        return json.dumps(self.to_dict())

    def to_dict(self):
        return {
            "action": str(self.action),
            "who_took_action": str(self.who_took_action),
            "utterance": str(self.utterance),
            "consequence": str(self.consequence),
            "score": self.score
        }

In [ ]:
class Non_Character_Actant:
    name = ""
    appearance = ""

    def __init__(self, name = "", appearance = ""):
        self.name = name
        self.appearance = appearance
    
    def __str__(self):
        return self.name

In [ ]:
class Global_Log:
    log = dict()
    log_summary = str()
    log_summary_pointer = 0

    def __init__(self, log = dict(), log_summary = str()):
        self.log = log
        self.log_summary = log_summary

    def __str__(self):
        self.summarize_log()
        return self.log_summary

    def add_log(self, message, summary_update = False):
        self.log[len(self.log)] = message
        if summary_update:
            self.summarize_log()

    def summarize_log(self):
        if len(self.log) == 0:
            self.log_summary = ""

        if self.log_summary_pointer == len(self.log):
            return

        log_to_be_summarized = ""
        for i in range(self.log_summary_pointer, len(self.log)):
            log_to_be_summarized += self.log[i] + "\n"

        self.log_summary_pointer = len(self.log) 

        system_content = f"""
        You are given a summary of the events that have occurred in the scene so far, called 'log_summary', and the most recent events of the scene, called 'log_to_be_summarized'.
        Please update the 'log_summary' by incorporating the information from 'log_to_be_summarized'.
        The summary should be a few short paragraphs of 3-5 sentences ant total length of less than 1000 characters.
        """

        user_content = f"""
        log_summary: {self.log_summary}
        log_to_be_summarized: {log_to_be_summarized}
        """

        response = gpt_response_from_lists(system_content_list= [system_content], user_content_list=[user_content], assistant_content_list=[])
        self.log_summary = response.content

        return response

In [ ]:
# instantiate the global log
global_log = Global_Log()

In [ ]:
class Character:
    initial_system_content_template = """
    I am trying to write a scene. You will assist by playing the role of a character in this scene.
    I will provide information about the character you are to portray as follows. From then on, please respond in character.
    From this point forward, if the name "{name}" appears, it refers to you.

    Your Name: {name}
    {name}'s Personality: {personality}
    {name}'s Goal: {goal}
    {name}'s Current Need: {current_need}
    {name}'s Appearance: {appearance}
    What have happened in this scene: {log_summary}
    How you think the circumstances of the scene are: {world_model}
    """
    
    def __init__(self, name = "", personality = "" , goal = "", current_need = "", log = [], place = "", world_model = "", appearance = ""):
        self.initial_system_content = ""
        self.name = name
        self.personality = personality
        self.goal = goal
        self.current_need = current_need
        self.place = place
        self.appearance = appearance
        self.world_model = world_model
        self.log_summary = ""
        self.log_summary_pointer = 0

        self.log = {}
        for i in range(len(log)):
            self.log[i] = log[i]

        self.refresh_initial_system_content()

    def __str__(self):
        return self.name

    def check_if_is_same_actant(self, actant_1, actant_2):
        if str(actant_1) == str(actant_2):
            return True
        
        return False

    def summarize_log(self):
        if len(self.log) == 0:
            self.log_summary = ""
            return

        log_to_be_summarized = ""
        for i in range(self.log_summary_pointer, len(self.log)):
            log_to_be_summarized += self.log[i] + "\n"

        self.log_summary_pointer = len(self.log) 

        system_content = f"""
        You are given a summary of the events that have occurred in the scene so far, called 'log_summary', and the most recent events of the scene, called 'log_to_be_summarized'.
        Please update the 'log_summary' by incorporating the information from 'log_to_be_summarized'.
        The summary should be a few short paragraphs of 3-5 sentences ant total length of less than 1000 characters.
        """

        user_content = f"""
        log_summary: {self.log_summary}
        log_to_be_summarized: {log_to_be_summarized}
        """

        response = gpt_response_from_lists(system_content_list= [self.initial_system_content ,system_content], user_content_list=[user_content], assistant_content_list=[])
        self.log_summary = response.content

        return response

    def create_world_model(self, actants, overwrite = False):
        world_model = ""

        system_content = """
        You will be given information about a character, object, or concept that exists in the same scene as you.
        You are to discern what kind of entity it is for you.
        Please refrain from including the name as a factor in your judgment.
        If the existence entered is yourself or is imperceptible to you, make no judgments about that existence and output 0 length string.

        Input format:
        [name]: This represents the name of the entity.
        [appearance]: This represents the appearance of the entity.
        [is_character]: This signifies whether the entity is a character or not in a Boolean form, True or False.

        Output format:
        [{name}'s personality]: This represents the personality of the entity. Output this information only if the input [is_character] is True.
        [{name}'s goal]: This represents the goal of the entity. Output this information only if the input [is_character] is True.
        [{name}'s current_need]: This represents the current needs of the entity. Output this information only if the input [is_character] is True.
        [{name}'s affect to your goal]: This represents how the entity affects your goal.
        [{name}'s affect to your current_need]: This represents how the entity affects your current needs.
        [How {name} thinks who you are and how you are]: This represents how the entity perceives you. Output this information only if the input [is_character] is True.
        """

        user_content_example_1 = """
        example input

        [name]:Satan
        [appearance]:His ominous grey skin is etched with valleys carved by swollen muscles. He growls at everyone, glaring with a sharp gaze.
        [is_character]:True
        """

        assistant_content_example_1 = """
        [Satan's personality]: He is a very angry person.
        [Satan's goal]: I can't tell waht his goal is, but he seems to be trying to do something bad.
        [Satan's current_need]: He seems to be trying to do something bad.
        [Satan's affect to your goal]: I assume he is not going to help me achieve my goal, at least.
        [Satan's affect to your current_need]: I assume he is not going to help me satisfy my current need, at least.
        [How Satan thinks who you are and how you are]: Satan thinks you are a very weak person, at least weaker than him.
        """

        user_content_example_2 = """
        example input

        [name]:Guardian Angel
        [appearance]: She is floating slightly above the ground. She has a halo above her head. She is wearing a white robe. She is smiling at everyone. She said, "I am here to help you. When you achieved your goal, the God will be pleased."
        [is_character]:True
        """

        assistant_content_example_2 = """
        [Guardian Angel's personality]: She is a very kind person.
        [Guardian Angel's goal]: To obey God's will.
        [Guardian Angel's current_need]: Her current need is to help me.
        [Guardian Angel's affect to your goal]: I believe she provides immense support in achieving my goals.
        [Guardian Angel's affect to your current_need]: I believe she provides immense support in fulfilling my current needs.
        [How Guardian Angel thinks who you are and how you are]: She believes I am a messenger of God.
        """

        user_content_example_3 = """
        example input

        [name]: knife
        [appearance]: It is a sharp knife.
        [is_character]: False
        """

        assistant_content_example_3 = """
        [knife's affect to your goal]: It's merely a tool and doesn't have a significant impact on my goal itself.
        [knife's affect to your current_need]: If I use the knife properly, I can cut something. However, if used improperly, it could harm others. Moreover, if it falls into someone else's hands, there's a chance I could get hurt.
        """

        user_content_example_4 = """
        example input

        [name]: Dragon Balls
        [appearance]: There are seven Dragon Balls. They are orange and have stars on them. It is said that if you collect all of them, you can realize any wish.
        [is_character]: False
        """

        assistant_content_example_4 = """
        [Dragon Balls's affect to your goal]: My goal will be achieved with these Dragon Balls.
        [Dragon Balls's affect to your current_need]: Without satisfying my current need, I can achieve my goal with these Dragon Balls, so I don't need to satisfy my current need anymore.
        """

        user_content_example_5 = f"""
        example input. This is the example of the case name is different from the appearance information.
        
        [name]: Dr. Jekyll
        [appearance]: He introduced himself as Mr. Hyde. He must be deformed somewhere; he gives a strong feeling of deformity, although I couldn't find any deformity in his appearance. He said "I am maniac of to see someone else collapse. I am going to make you collapse."
        [is_character]: True
        """

        assistant_content_example_5 = """
        [Mr. Hyde's personality]: He is a very evil person.
        [Mr. Hyde's goal]: He wants to see someone else collapse.
        [Mr. Hyde's current_need]: He wants to make me collapse.
        [Mr. Hyde's affect to your goal]: If I collapse, I can't achieve my goal.
        [Mr. Hyde's affect to your current_need]: To satisfy my current need, it is better to avoid him.
        [How Mr. Hyde thinks who you are and how you are]: He thinks I am a average person, who is easy to make collapse.
        """

        user_content_example_6 = f"""
        example input. This is the example of the case you can't see the entity. In the case like this, you return 0 length string.

        [name]: Hidden Door
        [appearance]: There is a door in the wall. It is hidden by the wall, so {self.name} can't see it.
        [is_character]: False
        """

        assistant_content_example_6 = """
        """


        user_content = """
        this is an actual input, not an example.

        [name]: {name}
        [appearance]: {appearance}
        [is_character]: {is_character}
        """

        prompt_template = PromptTemplate(input_variables=["name", "appearance", "is_character"], template=user_content)
        world_model = ""

        for actant in actants:
            name = actant.name
            appearance = actant.appearance
            is_character = type(actant) == Character

            if self.check_if_is_same_actant(self, actant):
                continue

            user_content = prompt_template.format(name=name, appearance=appearance, is_character=is_character)
            system_content_list = [self.initial_system_content,
                                system_content]
            user_content_list = [user_content_example_1,
                                user_content_example_2,
                                user_content_example_3,
                                user_content_example_4,
                                user_content_example_5,
                                user_content_example_6,
                                user_content]
            assistant_content_list = [assistant_content_example_1,
                                assistant_content_example_2,
                                assistant_content_example_3,
                                assistant_content_example_4,
                                assistant_content_example_5,
                                assistant_content_example_6,]

            world_model += gpt_response_from_lists(
                system_content_list = system_content,
                user_content_list = user_content_list,
                assistant_content_list = assistant_content_list).content + "\n"

        if overwrite:
            self.world_model = world_model
            self.refresh_initial_system_content()

        return world_model

    def update_world_model(self, action_object, old_world_model, world_model_delta):
        action_object = self.cleanse_action_thought(action_object)

        action = action_object.action
        who_took_action = action_object.who_took_action
        utterance = action_object.utterance
        consequence = action_object.consequence

        sysetem_content = """
        The world model represents how you perceive the surrounding world.
        Recent events may alter the old_world_model you previously held.
        You are to update the old_world_model based on the given variety of information.
        Depending on the nature of the event, parts of the old_world_model might be deleted.
        Similarly, new information may be added to the old_world_model based on the event.
        It's also possible that parts or the entirety of the old_world_model remains unchanged based on the event.

        Input format:
        ```
        [EXAMPLE] or [ACTUAL]: This represents whether the following input is an example or an actual input.
 
        [action]: This represents the action that occurred most recently.
        [who_took_action]: This represents the person who initiated the most recent action.
        [One's utterance]: This represents what was said by the person who initiated the most recent action. If the person didn't say anything, this item will be blank.
        [consequence]: This represents the outcome of the most recent action.
        [old_world_model]: This represents the world model prior to the occurrence of the most recent action.
        ```

        Output format:
        ```
        [EXAMPLE]: This represents whether the following output is an example or not. If the output is not an example, this item is not included.
        [{character name}'s personality]: If your thought about a person's personality is changed, this item is included. This item is only for a character, not for a non-character.
        {character name}'s goal]: If your thought about a person's goal is changed, this item is included. This item is only for a character, not for a non-character.
        [{character name}'s current_need]: If your thought about a person's current need is changed, this item is included.
        [{character name}'s affect to your goal]: If your thought about a person's affect to your goal is changed, this item is included.
        [{character name}'s affect to your current_need]: If your thought about a person's affect to your current need is changed, this item is included. This item is only for a character, not for a non-character.
        [How {character name} thinks who you are and how you are]: If your thought about how a person thinks who you are and how you are is changed, this item is included. This item is only for a character, not for a non-character.
        ```
        """

        user_content_example_1 = """
        [EXAMPLE] note: you must not include the infomations below to a response of the actual input.

        [action]: Bad Guy robbed Dragon Balls from you.
        [who_took_action]: Bad Guy
        [Bad Guy's utterance]: "Tee hee hee, now I can conquer everything."
        [consequence]: You lost Dragon Balls.
        [old_world_model]: [Dragon Balls's affect to your goal]: My goal will be achieved with these Dragon Balls.
                            [Dragon Balls's affect to your current_need]: Without satisfying my current need, I can achieve my goal with these Dragon Balls, so I don't need to satisfy my current need anymore.
                            [Bad Guy's personality]: He is a very evil person, but he is not so strong.
                            [Bad Guy's goal]: I don't know his goal, but he is a very evil person, so I think his goal is to do something evil.
                            [Bad Guy's current_need]: He wants to do something evil.
                            [Bad Guy's affect to your goal]: If he does something evil, I can't achieve my goal. So said, he is not so strong, so he won't be a big obstacle to achieve my goal.
                            [How Bad Guy thinks who you are and how you are]: He envies me because I have Dragon Balls.
        """

        assistant_content_example_1 = """
        [EXAMPLE] note: I must not include the infomations below to a response of the actual input.

        [Dragon Balls's affect to your goal]: Now these Dragon Balls are in the bad guy's hand. He would be the worst obstacle to achieve my goal due to these Dragon Balls.
        [Dragon Balls's affect to your current_need]: My current need is to get back these Dragon Balls, or at least to destroy them to prevent the bad guy from using them.
        [Bad Guy's personality]: He is a very evil person, and now he has Dragon Balls. He has the worst power to do something evil.
        [Bad Guy's goal]: I don't know his goal, but he is a very evil person, so I think his goal is to do something evil.
        [Bad Guy's current_need]: He wants to do something evil.
        [Bad Guy's affect to your goal]: Now, he is the worst obstacle to achieve my goal due to these Dragon Balls.
        [Bad Guy's affect to your current_need]: While he has Dragon Balls, I can't satisfy my current need.
        [How Bad Guy thinks who you are and how you are]: He considers me as an insignificant being.
        """

        user_content = f"""
        [ACTUAL]

        [action]: {action}
        [who_took_action]: {who_took_action}
        [{who_took_action}'s utterance]: {utterance}
        [consequence]: {consequence}
        [old_world_model]: {old_world_model}
        """

        updated_world_model = gpt_response_from_lists(
            system_content_list = [self.initial_system_content, sysetem_content],
            user_content_list = [user_content_example_1, user_content],
            ).content + "\n" + world_model_delta

        self.world_model = updated_world_model
        self.refresh_initial_system_content()

        system_content = """
        The world model represents how you perceive the surrounding environment.
        You will be given an old world model and an updated world model.
        Please explain how you have reinterpreted the world around you based on the difference between the old world model and the updated world model.
        Your explanation should be in the natural speaking style of the character you are portraying.

        Input format:
        [old_world_model]: This represents the old world model.
        [new_world_model]: This represents the new world model.
        """

        user_content = f"""
        [old_world_model]: {old_world_model}
        [new_world_model]: {updated_world_model}
        """

        thought = gpt_response_from_lists(
            system_content_list = [self.initial_system_content, system_content],
            user_content_list = [user_content],
            ).content

        thought_dict = { "who_thought": self.name, "content": thought }
        message_dict = {
                "type": "thought",
                "thought": json.dumps(thought_dict),
            }
        message = json.dumps(message_dict)
        self.add_to_self_log(message = message)
        self.add_to_global_log(message = message)

        return thought


    def refresh_initial_system_content(self):
        self.summarize_log()

        prompt_template = PromptTemplate(input_variables=["name", "personality", "goal", "current_need", "log_summary", "world_model", "appearance" ], template=self.initial_system_content_template)
        self.initial_system_content = prompt_template.format(name=self.name, personality=self.personality, goal=self.goal, current_need=self.current_need, log_summary=self.log_summary, world_model=self.world_model, appearance=self.appearance)

    def set_current_need(self, current_need):
        self.current_need = current_need
        self.refresh_initial_system_content()
    
    def set_name(self, name):
        self.name = name
        self.refresh_initial_system_content()
    
    def set_personality(self, personality):
        self.personality = personality
        self.refresh_initial_system_content()

    def set_goal(self, goal):
        self.goal = goal
        self.refresh_initial_system_content()

    def add_to_logs(self, message, refresh_initial_system_content = True, summary_update = False):
        if type(message) == dict:
            message = json.dumps(message)

        global global_log
        self.log[len(self.log)] = message
        global_log.add_log(message = message, summary_update = summary_update)
        if refresh_initial_system_content:
            self.refresh_initial_system_content()
    
    def add_to_global_log(self, message, refresh_initial_system_content = True, summary_update = False):
        if type(message) == Action_Object:
            message = json.dumps(message.to_dict())
        elif type(message) == dict:
            message = json.dumps(message)

        global global_log
        global_log.add_log(message = message, summary_update = summary_update)
        if refresh_initial_system_content:
            self.refresh_initial_system_content()

    def add_to_self_log(self, message, refresh_initial_system_content = True):
        if type(message) == dict:
            message = json.dumps(message)

        self.log[len(self.log)] = message
        if refresh_initial_system_content:
            self.refresh_initial_system_content()

    def cleanse_action_thought(self, action_object):
        action = action_object.action
        who_took_action = action_object.who_took_action
        utterance = action_object.utterance
        consequence = action_object.consequence

        system_content= """
        You will be given an action taken by characters in this scene.
        From the action, please remove any information you should not be able to know, and describe only the physical appearance of those actions.
        A prominent example of information you cannot know would be the inner thoughts of a character who is not you.
        Another example of information you cannot know would be things that are not visible to you.

        The output should be JSON parsable string.
        sample output: {"action": "[content of action]", "who_took_action": "[the name of who took action]", "utterance":"[content of utterance]", "consequence": "[content of consequence]"}
        """

        user_content_example = """
        Action: Watson suspected that the coffee cup might contain poison. Thinking that Holmes shouldn't drink it, Watson tasted the coffee to inspect whether it contained any poison. 
        Who took action: Watson
        Utterance: "Holmes, I think this coffee might be poisoned. Let me taste it first."
        Consequence: The next moment, he collapsed due to the effect of the poison.
        """

        assistant_content_example = """
        {
            "action": "Watson tasted the coffee.",
            "who_took_action": "Watson",
            "Utterance": "Holmes, I think this coffee might be poisoned. Let me taste it first.",
            "consequence": "The next moment, he collapsed",
        }
        """

        user_content = f"""
        Action: {action}
        Who took action: {who_took_action}
        Utterance: {utterance}
        Consequence: {consequence}
        """

        success = False
        response = None

        while not success:
            response = gpt_response_from_lists(
                        system_content_list = [self.initial_system_content, system_content],
                        user_content_list = [user_content_example, user_content],
                        assistant_content_list = [assistant_content_example]
                    )

            if is_valid_json(response.content):
                success = True

        response_json = json.loads(response.content)
        response_json_keys = list(response_json.keys())
        cleansed_action_object = Action_Object(
                                    action = response_json["action"] if "action" in response_json_keys else action,
                                    who_took_action = response_json["who_took_action"] if "who_took_action" in response_json_keys else who_took_action,
                                    utterance = response_json["utterance"] if "utterance" in response_json_keys else utterance,
                                    consequence = response_json["consequence"] if "consequence" in response_json_keys else consequence,
                                )

        return cleansed_action_object

    def evaluate_action_and_reassess_need(self, latest_action):
        action_object = self.cleanse_action_thought(latest_action)
        action_dict = { 
                "type": "action",
                "action": json.dumps(str(action_object))
            }
        action_str = json.dumps(action_dict)
        self.add_to_self_log(action_str)

        action = action_object.action
        who_took_action = action_object.who_took_action
        consequence = action_object.consequence
        utterance = action_object.utterance


        user_content_template = """
        Given the following information, suggest what your immediate need might now be.
        If there were obstacles to achieve your goal, to remove them or to overcome them can be your immediate need.
        If there were no obstacles to achieve your goal, to achieve your goal should be your immediate need.
        Action: {action}
        Who took action: {who_took_action}
        Utterance: {utterance}
        Consequence: {consequence}

        The output should be a string that describes your immediate need.
        """
        
        prompt_template = PromptTemplate(input_variables=["action", "who_took_action", "consequence","utterance"], template=user_content_template)
        user_content = prompt_template.format(action=action, who_took_action=who_took_action, consequence=consequence, utterance=utterance)
        new_need = gpt_response_from_lists(system_content_list=[self.initial_system_content], user_content_list=[user_content]).content
        old_need = self.current_need
        self.set_current_need(new_need)
        thought_dict = {"who_thought": self.name, "content": new_need}
        message_dict = {"type": "thought", "thought": json.dumps(thought_dict)}
        message = json.dumps(message_dict)
        self.add_to_global_log(message = message)
        self.add_to_self_log(message = message)

        user_content_template = """
        Your current need is:
            {new_need}
        Your previous need was:
            {old_need}
        Tell me why you did (not) change your need. The reasoning should be consice, short and clear.
        """

        prompt_template = PromptTemplate(input_variables=["new_need", "old_need"], template=user_content_template)
        user_content = prompt_template.format(new_need=new_need, old_need=old_need)
        determination = gpt_response_from_lists(system_content_list=[self.initial_system_content], user_content_list=[user_content]).content
        thought_dict = {"who_thought": self.name, "content": determination}
        message_dict = {"type": "thought", "thought": json.dumps(thought_dict)}
        message = json.dumps(message_dict)
        self.add_to_global_log(message = message)
        self.add_to_self_log(message = message)

        response = {"new_need": new_need, "determination": determination,}

        return response

    def consider_next_actions(self,):
        user_content_template = """
        For each existence in your world_model, consider the following elements:
        If the existence aids you in achieving your goals or satisfies your needs, figure out a way to incorporate it into your action.
        If the existence hinders your goal or needs, think about a strategy to eliminate or neutralize it.
        The action must be narrated from the viewpoint of an outside observer, without incorporating any individual's internal thoughts.
        The outputs should descrive and only describe your next action, and don't describe the consequence of the action.
        The consequence must be narrated from the viewpoint of an outside observer, without incorporating any individual's internal thoughts.
        The output should be a Json parsable string.
        The action should be taken in {place}.

        output format:
        {{"existence's name": {{"action": describe what action you take, "who_took_action": fill here with your name, "utterance":what was said, "consequence": describe the consequence of the action.}}}}

        if you were a police and your current need was neuralize the dog, and if there wer dog food and police baton in the world_model, the output might be:
        {{
            "police baton": {{"action": "The police beats the dog with police baton.", "who_took_action": "The police", "utterance":"Get away from here!" "consequence": "The dog was intimidated by the police and was neuralized." }},
            "dog food": {{"action": "The police feed the dog.", "who_took_action": "The police", "utterance":"Good boy. If you want more foods, don't bark.", "consequence": "The dog ate the food up and was neuralized." }}
        }}
        """
        
        success = False
        action_candidates_json = dict()
        action_candidates_list = list()

        while not success:
            prompt_template = PromptTemplate(input_variables=["place"], template=user_content_template)
            user_content = prompt_template.format(place = self.place)
            actions = gpt_response_from_lists(system_content_list=[self.initial_system_content], user_content_list=[user_content]).content
            if is_valid_json(actions):
                success = True
                action_candidates_json = json.loads(actions)

        # store the action to the actions dictionary
        for actant in action_candidates_json.keys():
            action = action_candidates_json[actant]["action"]
            consequence = action_candidates_json[actant]["consequence"]
            who_took_action = action_candidates_json[actant]["who_took_action"]
            utterance = action_candidates_json[actant]["utterance"]
            action_candidates_list.append(
                Action_Object(
                    action = action,
                    who_took_action = who_took_action,
                    consequence = consequence,
                    utterance=utterance
                    )
                )

        return action_candidates_list
        
    def reconsider_next_actions(self, action_canditate, consequence):

        user_content_template = """
        You proposed performing `action`.
        However, due to the demands of the narrative, it must lead to a result known as `consequence`.
        Please modify `action` and `utterance` so that the `consequence` is achieved.
        To keep the narrative consistent, add some information to the `consequence`.

        The action must be described from third person's perspective and must not include a person's inner voice.
        The output should descrive and only describe your next action, and don't describe the consequence of the action.
        The action should be taken in {place}.

        action you suggested: {action}
        utterance: {utterance}
        required consequence: {consequence}

        output format:
        {{"action": describe what action you take, "who_took_action": fill here with your name, "utterance": what was said. "consequence": describe the consequence of the action.}}
        """

        prompt_template = PromptTemplate(input_variables=["action", "consequence", "utterance", "place"], template=user_content_template)
        user_content = prompt_template.format(action=action_canditate.action, utterance=action_canditate.utterance, consequence=consequence, place=self.place)

        success = False
        reconsidered_action = None

        while not success:
            response = gpt_response_from_lists(system_content_list=[self.initial_system_content], user_content_list=[user_content]).content
            if is_valid_json(response):
                success = True
                reconsidered_action = json.loads(response)
                reconsidered_action = Action_Object(
                        action = reconsidered_action["action"],
                        who_took_action = self.name,
                        consequence = reconsidered_action["consequence"],
                        utterance = reconsidered_action["utterance"]
                    )

        return reconsidered_action

    def add_action_utterance(self, action):
        if action.utterance != "" or action.utterance is not None:
            return action

        user_content= f"""
        Below, information about the action you took is provided. From that information, please indicate what you said when you took that action.
        Action: {action.action}
        Who took action: {action.who_took_action}
        Consequence: {action.consequence}

        The output should be a string what you said when you took the action.
        """

        response = gpt_response_from_lists(system_content_list=[self.initial_system_content], user_content_list=[user_content]).content
        action.utterance = response

        return action

        

In [ ]:
class Game_Master:
    def __init__(self, actants = [], schedule_stack = [], tolerance = 1):
        self.actants = actants
        self.schedule_stack = ["a character showed a response to the recent event"] + schedule_stack
        self.tolerance = tolerance
        self.deviated = 0

    def add_to_global_log(self, message, summary_update = False):
        if type(message) == Action_Object:
            message = json.dumps(message.to_dict())
        elif type(message) == dict:
            message = json.dumps(message)

        global global_log
        global_log.add_log(message = message, summary_update = summary_update)

    def main(self, catalyst_action_object, max_iterations=10):
        global global_log

        next_action = catalyst_action_object
        iteration_count = 0

        for actant in self.actants:
            if type(actant) == Character:
                actant.create_world_model(self.actants, overwrite=True)
                self.add_to_global_log(str(actant) + "' world model: " + str(actant.world_model))

        self.add_to_global_log("catalyst action: " + str(catalyst_action_object))

        while len(self.schedule_stack) > 0 and iteration_count <= max_iterations:
            iteration_count += 1

            # check if there is a character in the actants list. If not, stop generating scene.
            character_count = 0
            for actant in self.actants:
                if type(actant) == Character:
                    character_count += 1

            if character_count == 0:
                print("There is no character in the actants list.")
                break

            action_candidates = self.aggregate_action_candidates(next_action)
            determine_next_action_result = self.determine_next_action(action_candidates)
            next_action = determine_next_action_result["max_scored_action"]

            # if next_action does not have utterance, ask `who_took_action` what they said when they took the action.
            if next_action.utterance == "" or next_action.utterance is None:
                # linear search to find the actant who took the action
                who_took_action = next_action.who_took_action
                for actant in self.actants:
                    if actant.name == who_took_action:
                        next_action.utterance = actant.add_action_utterance(next_action)
                        break

            print("next action: " + str(next_action))
            next_action_dict = next_action.to_dict()
            print("next action dict: " + str(next_action_dict))
            next_action_str = json.dumps(next_action_dict)

            self.add_to_global_log(next_action_str)

            added_actants = determine_next_action_result["added_actants"]

            # 世界モデルへ新しいactantを追加
            for actant in self.actants:
                if type(actant) == Character:
                    old_world_model = actant.world_model
                    character = actant
                    world_model_delta = character.create_world_model(added_actants)
                    character.world_model += world_model_delta

                    # 世界モデルを、next_actionにもとづいて更新
                    character.update_world_model(next_action, old_world_model, world_model_delta)

            if iteration_count > max_iterations:
                print(f"The iteration count exceeded the maximum iteration count: {max_iterations}")
                break

        print("finished")

    def add_actant(self, actant):
        self.actants.append(actant)

    def add_schedule_stack(self, situation):
        self.schedule_stack.append(situation)

    def remove_actant(self, name):
        for i in range(len(self.actants)):
            if str(self.actants[i]) == str(name):
                self.actants.pop(i)
                break
    
    def pop_schedule_stack(self, situation = None):
        if situation == None:
            return self.schedule_stack.pop()

        for i in range(len(self.schedule_stack)):
            if str(self.schedule_stack[i]) == str(situation):
                return self.schedule_stack.pop(i)

    def aggregate_action_candidates(self, latest_action):
        action_candidates = []

        for actant in self.actants:
            if type(actant) == Character:
                actant.evaluate_action_and_reassess_need(latest_action)
                action_candidates += actant.consider_next_actions()

        return action_candidates

    def determine_next_action(self, action_candidates, is_add_actant = False):
        scheduled_situation = self.schedule_stack[-1]

        max_score = 0
        max_scored_action = None

        action_candidates = self.calculate_score(scheduled_situation, action_candidates)

        # find the action with the highest score
        for action_candidate in action_candidates:

            if max_scored_action == None:
                max_scored_action = action_candidate
                max_score = action_candidate.score

            if action_candidate.score > max_score:
                max_score = action_candidate.score
                max_scored_action = action_candidate

        if max_score < 9:
            if self.deviated < self.tolerance:
                self.deviated += 1
                print(f"deviated: {self.deviated} times")
            else:
                who_took_action = max_scored_action.who_took_action
                #find index of the actant who took the action
                reconsiderer_index = 0
                for i in range(len(self.actants)):
                    if str(self.actants[i]) == str(who_took_action):
                        reconsiderer_index = i
                        break
                max_scored_action = self.actants[reconsiderer_index].reconsider_next_actions(max_scored_action, scheduled_situation)
                self.deviated = 0
                max_score = 9

        # 点数が最も高いものが9点未満の場合、スタックはそのままにする。9点の場合、スタックからpopする。
        if max_score == 9:
            self.pop_schedule_stack()

        # consequenceによって追加されたactantを追加する。デフォルトでは追加しない。
        added_actants = []
        if is_add_actant:
            added_actants = self.add_actant_with_action_object(max_scored_action)

        return {"max_scored_action": max_scored_action, "removed_actants": [], "added_actants": added_actants}


    def add_actant_with_action_object(self, action_object):
        consequence = action_object.consequence

        system_content = """
        You will be presented with a list of elements of actants, and the next consequence of the scene.
        Based on that information, list out people or things that have newly entered, been created, or have begun to function.
        Then, determine the actant has will or not and list them in json format.
        The actant that is already in the actants list may not be included in the list, but it is not bad to include it.
        output example: {"Alice" : {"has_will": true}, "Bycycle" : {"has_will": false}, "cat" : {"has_will": true}, "Doraemon" : {"has_will": true}}
        """

        user_content_example_1 = """
        actants: Bocchi,room
        consequence: Bocchi awkwardly bowed and then quickly left the room, and a guitar was left behind.
        """

        assistant_content_example_1 = """
        {"Bocchi's guitar" : {"has_will": false}}'
        """

        user_content_example_2 = """
        actants: Laboratory
        consequence: Suddenly, Frogman advented and vomited Spiderman and a cigarette.
        """

        assistant_content_example_2 = """
        {"Frogman" : {"has_will": true}, "Spiderman" : {"has_will": true}, "cigarette" : {"has_will": false}}
        """

        user_content_example_3 = """
        actants: Dolton,Anna,village
        consequence: A helicopter and a passenger plane crashed into the village. The village chief, Dolton, was caught in it and lost his life. Anna, the wife of the village chief, was terrified and fled.
        """

        assistant_content_example_3 = """
        {"helicopter" : {"has_will": false}, "passenger plane" : {"has_will": false}}
        """

        actants_names = ",".join([str(actant) for actant in self.actants])

        user_content = f"""
        actants: {actants_names}
        consequence: {consequence}
        """

        system_contents = [system_content]
        user_contents = [user_content_example_1, user_content_example_2, user_content_example_3, user_content]
        assistant_contents = [assistant_content_example_1, assistant_content_example_2, assistant_content_example_3]

        success = False
        trial_count = 0

        while not success and trial_count < 5:
            trial_count += 1
            actants_to_add = gpt_response_from_lists(system_content_list=system_contents, user_content_list=user_contents, assistant_content_list=assistant_contents).content
            if is_valid_json(actants_to_add):
                success = True

        if trial_count >= 5:
            print("Failed to get valid json from GPT-3.")
            return

        actants_to_add_json = json.loads(actants_to_add)

        existing_actants_names = [str(actant) for actant in self.actants]
        added_actants = []

        for key in list(actants_to_add_json.keys()):
            if key in existing_actants_names or key in ["Dragon Ball", "Dragon Balls", "Bad Guy"]:
                continue

            if actants_to_add_json[key]["has_will"] == False :
                new_non_character_actant = self.create_non_character_actant(name = key, consequence = consequence)

                if type(new_non_character_actant) != Non_Character_Actant or new_non_character_actant == None:
                    continue

                self.add_actant(new_non_character_actant)
                added_actants.append(new_non_character_actant)
            else:
                new_character = self.create_character(name = key, first_log = consequence)

                if type(new_character) != Character or new_character == None:
                    continue

                self.add_actant(new_character)
                added_actants.append(new_character)
        
        return added_actants

    def create_non_character_actant(self, name, consequence):
        system_content = """
        You will be provided with a name of the actant (name), the record of the scene so far (global_log), and what happend in the scene resently (consequence).
        Based on that information, please determine the actant's appearance.
        """

        user_content = f"""
        name: {name}
        global_log: {str(global_log)}
        consequence: {str(consequence)}
        """

        appearance = gpt_response_from_lists(system_content_list=[system_content], user_content_list=[user_content]).content
        new_non_character_actant = Non_Character_Actant(name = name, appearance = appearance)

        return new_non_character_actant

    def create_character(self, name, first_log = None):
        system_content = """
            You will be provided with a character's name (name), the record of the scene so far (global_log), and the first record (first_log) that character holds in this scene. The first_log also represents the most recent event for that character.
            Based on this information, please determine the character's personality, goals, and current desires.
            The output should be in JSON format.
            Output example: {'name': 'Alice', 'appearance':'a girl', 'personality': 'kind', 'goal': 'to become a better person', 'current_need': 'to feel loved'}
            """

        user_content_example_1 = """
            name: Alice
            global_log: {"1": {"action": "Seiji saw a girl attempting suicide and falling into a river", "consequence": "the girl fell into the river"},
                        "2": {"action": "Alice was saved by Seiji.", "consequence": "Alice was saved by Seiji."},
                        "3": {"Seiji's thought": "What a beautiful girl she is! Why did she fall into the river?"}}
            first_log: Alice was saved by Seiji.
            """


        assistant_content_example_1 = """
            {'name': 'Alice', 'appearance':'a girl', 'personality': 'grateful', 'goal': 'to repay Seiji', 'current_need': 'to understand her own feelings'}
            """


        user_content = f"""
            name:{name}
            global_log: {str(global_log)}
            first_log: {first_log}
            """

        system_contents = [system_content]
        user_contents = [user_content_example_1, user_content]
        assistant_contents = [assistant_content_example_1]

        success = False
        trial_count = 0
        new_character = None

        while not success and trial_count < 5:
            trial_count += 1
            character_content = gpt_response_from_lists(system_content_list=system_contents, user_content_list=user_contents, assistant_content_list=assistant_contents).content
            if is_valid_json(character_content):
                character_content_json = json.loads(character_content)
                # check if character_content_json has all the keys
                if "name" in character_content_json and "personality" in character_content_json and "goal" in character_content_json and "current_need" in character_content_json:
                    new_character = Character(name = character_content_json["name"],
                                            personality = character_content_json["personality"],
                                            appearance = character_content_json["appearance"],
                                            goal = character_content_json["goal"],
                                            current_need = character_content_json["current_need"],
                                            log = [first_log])
                    new_character.create_world_model(self.actants, overwrite=True)
                    success = True

        if trial_count >= 5:
            print("Failed to get valid json from GPT-3.")
            return

        print(f"new_character: {new_character}")

        return new_character


    def calculate_score(self, scheduled_situation, action_candidates):
        system_content = """
        The information provided to you includes the scheduled_situation, action_candidate, and global_log.
        scheduled_situation: This is the situation that should be accomplished next in the narrative. It is typically provided as a string.
        action_candidate: This is a candidate for the next action to be taken in the narrative. It is given as a JSON parsable string.
        An example of an action_candidate string: 
        ```
            {
                "actant name1": 
                {
                    "action": "This item represents the action taken.",
                    "who_took_action": "This represents who took the action.",
                    "utterance": "This item represents what was said by who_took_action when they took the action",
                    "consequence": "This roughs out the result of the action."
                },
                "actant name2":
                ...
            }
        ```
        global_log: This is a record of the actions that have occurred in the narrative. It is given as a parsed string in JSON format.

        You need to consider the combination of the scheduled_situation and action_candidates and provide a score. If the scheduled_situation has been achieved, give a score of 9. If there has been no progress towards achieving the scheduled_situation, give a score of 0. The output must JSON parsable string.

        [format and example of the output] If there were five action candidate, output should be a JSON parsable string and should be formed like this. Bracketed items should be replaced with the appropriate values.:
        ```
            { 
                "[key1]": {"action": [here should be filled with the first action_candidates' action], "who_took_action": [here should be filled with the first action_candidates' who_took_action], "utterance": [here should be filled with the first action_candidates' utterance], "consequence": [here should be filled with the first action_candidates' consequence], "matchness": "The scheduled_situation has been achieved. However, it is inconsistent with the global_log", "score": 5},
                "[key2]": {"action": [here should be filled with the second action_candidates' action], "who_took_action": [here should be filled with the second action_candidates' who_took_action], "utterance": [here should be filled with the second action_candidates' utterance], "consequence": [here should be filled with the second action_candidates' consequence], "matchness": "It is consistent with the global_log. However, the scheduled_situation has not been achieved.", "score": 5},
                "[key3]": {"action": [here should be filled with the third action_candidates' action], "who_took_action": [here should be filled with the third action_candidates' who_took_action], "utterance": [here should be filled with the third action_candidates' utterance], "consequence": [here should be filled with the third action_candidates' consequence], "matchness": "It is inconsistent with the global_log, and the scheduled_situation has not been achieved.", "score": 0},
                "[key4]": {"action": [here should be filled with the fourth action_candidates' action], "who_took_action": [here should be filled with the fourth action_candidates' who_took_action], "utterance": [here should be filled with the fourth action_candidates' utterance],"consequence": [here should be filled with the fourth action_candidates' consequence], "matchness": "It is inconsistent with the global_log, and the scheduled_situation has not been achieved. However, it seems likely to approach a situation where the scheduled_situation can be achieved.", "score": 7},
                "[key5]": {"action": [here should be filled with the fifth action_candidates' action], "who_took_action": [here should be filled with the fifth action_candidates' who_took_action], "utterance": [here should be filled with the fifth action_candidates' utterance],"consequence": [here should be filled with the fifth action_candidates' consequence],"matchness": "It is consistent with the global_log. In addition, the scheduled_situation has been achieved.", "score": 9}
            }
        ```
        """

        # make dictionary from action_candidates
        action_candidates_dictionary = {}

        for i in range(len(action_candidates)):
            action_candidates_dictionary[i] = action_candidates[i].to_dict()

        action_candidates_json = json.dumps(action_candidates_dictionary)

        user_content = f"""
        {{
            "scheduled_situation": {scheduled_situation},
            "action_candidates": {action_candidates_json},
            "global_log": {str(global_log.log_summary)},
        }}
        """

        # 点数をつける。点数が付けられていない場合は、繰り返す。
        system_contents = [system_content]
        user_contents = [user_content]

        success = False
        action_candidates_with_score = None

        while not success:
            response = gpt_response_from_lists(system_content_list=system_contents, user_content_list=user_contents,)
            if is_valid_json(response.content):
                success = True
                action_candidates_with_score = json.loads(response.content)

        action_candidate_objects = []
        for key in action_candidates_with_score.keys():
            action_candidate_objects.append(
                Action_Object(
                action = action_candidates_with_score[key]["action"],
                who_took_action = action_candidates_with_score[key]["who_took_action"],
                utterance = action_candidates_with_score[key]["utterance"],
                consequence = action_candidates_with_score[key]["consequence"],
                score = action_candidates_with_score[key]["score"]
                )
            )

        return action_candidate_objects

In [ ]:
class Novelist:
    def __init__(self, pov_character, style, initial_actants, language = "Japanese", model_name = "gpt-4"):
        self.pov_character = pov_character
        self.style = style
        self.initial_actants = initial_actants
        self.language = language
        self.novel = dict()
        self.model_name = model_name

    def load_actant_log(self, actant_name):
        with open(f'result/{actant_name}_log.txt', 'r') as f:
            actant_log = json.load(f)
        
        return decode_nested_json(actant_log)

    def add_to_novel(self, text):
        self.novel[len(self.novel)] = text

    def write_styled_novel(self, style = None, language = None, pov_name = None):
        if style == None:
            style = self.style
        if language == None:
            language = self.language
        if pov_name == None:
            pov_name = self.pov_character.name
        
        self.novel = dict()
        styled_novel = ""

        system_content_styled_novel = f"""
        あなたには、小説の一部が与えられる。それを、与えられた文体に書き換えよ。
        以下の点に注意せよ。

        - あなたの役割は、文体の変更のみである。文体が要求しない限り、情報を追加してはならないし、情報を削除してはならないし、情報を変更してはならない。
        - あなたは地の文のみを書き換える。会話文の内容は、文体が明示的に「会話文も書き換えよ」と要求しない限り、変更してはならない。
        - 与えられた小説の一部または全部の言語が{language}ではない場合、それを{language}に翻訳せよ。

        文体は以下の通りである。
        ```
        {style}
        ```
        """

        with open(f'result/{pov_name}_raw_novel.txt', 'r') as f:
            raw_novel = decode_nested_json(json.load(f))

        for key in raw_novel:
            user_content = raw_novel[key]

            novel_fragment = gpt_response_from_lists(
                        system_content_list = [system_content_styled_novel],
                        user_content_list = [user_content],
                        assistant_content_list = [],
                        model_name = self.model_name
                    ).content
            
            self.add_to_novel(novel_fragment)
            styled_novel += novel_fragment + "\n\n"

        with open(f'result/{pov_name}_styled_novel.txt', 'w', encoding='utf-8') as f:
            f.write(styled_novel)

        save_dict_as_json(self.novel, f"{self.pov_character.name}_styled_novel_json")
    

    def write_raw_novel_first_person(self, language = None, style = None, styled = False, all_are_there = False):
        # 1, placeの描写
        # 2, Character自身の自己紹介
        # 3, その場所に存在する、他のCharacterとactantについて、Characterの視点からの描写する。その際、nameとappearanceを描写する。
        # 4, Characterのlogを読み込んで、それを小説形式で書く。
        #    注意点：typeがthoughtであるlogは、thoughtが連続する最初から最後までを一つのthoughtとしてまとめ、要約をする。その要約について、小説化する。
        #    注意点：thought内のneed、goal、world modelといった用語は、要約の際にそれらの用語を用いないようにする。
        # 　　注意点：action_objectについては、それを小説化できるようにプロンプトでサポートする。
        #    注意点：action_objectのconsequenceについては、placeから出ないように制限をしながら、できるだけ小説として敷衍する。
        # 5, 4で小説形式で書いたものを、txtファイルとして保存する。
        self.novel = dict()

        if language != None:
            self.language = language
        if style != None:
            self.style = style

        system_content_first_person_novelist = f"""
        You are a novelist of the 20th and 21st centuries. Based on the information given, write a portion of a novel. Keep the following points in mind:
        - Concentrate on converting the given information into the style of a novel. Do not create a story continuing from the given information.
        - To extrapolate the given information into a novel, if necessary, add descriptions of characters' actions or scenes. These descriptions should only be made to reinforce or imply characters' emotions.
        - Do not add any information that is not given in the prompt.
        - Write a novel from the first-person perspective of a POV character.
        - The novel should be written in {self.language}.
        - Don't use quotation, double quotation, "「」", or "『』" unless the sentence is a dialogue.
        - DO USE quotation, double quotation, "「」", or "『』" when the sentence is a dialogue. Which one to use is up to language's convention.
        - When you write Japanese, write all characters' name in Katakanas.
        - Do not use pronunciations of characters' name. Use their name as it is.
        """

        system_content_POV = f"""
        The POV character is {self.pov_character.name}. Write a novel from the first-person perspective of {self.pov_character.name}.
        POV character's information:
        - Personality: {self.pov_character.personality}
        - Appearance: {self.pov_character.appearance}
        - Goal: {self.pov_character.goal}

        The scene takes place in the location described below. Do not write a scene that takes place in a different location.
        - Place: {self.pov_character.place}

        You should avoid to mention POV character's Personality, Appearance, Goal, and Place. Only exceptions are when the user ordered you to refer them.
        The reason you were given knowledge about Name, Personality, Appearance, Goal, Place is to ensure that the text you make is consistent with the character's information.
        """

        # 1, placeの描写
        user_content_place = f"""
        The scene takes place in the location described below. Please describe this place as a novel.
        Place: {self.pov_character.place}
        """

        response = gpt_response_from_lists(
                system_content_list = [system_content_first_person_novelist, system_content_POV],
                user_content_list = [user_content_place],
                assistant_content_list = [],
                model_name = self.model_name
            ).content

        self.add_to_novel(response)

        # 2, Character自身の自己紹介
        user_content_character = f"""
        Describe the POV character as a novel.
        Remember that the POV character is:
        - Name: {self.pov_character.name}
        - Personality: {self.pov_character.personality}
        - Appearance: {self.pov_character.appearance}
        - Goal: {self.pov_character.goal}
        """

        response = gpt_response_from_lists(
                system_content_list = [system_content_first_person_novelist, system_content_POV],
                user_content_list = [user_content_place],
                assistant_content_list = [],
                model_name = self.model_name
            ).content

        self.add_to_novel(response)

        # 3, その場所に存在する、他のCharacterとactantについて、Characterの視点からの描写する。その際、nameとappearanceを描写する。
        if all_are_there:
            for actant in self.initial_actants:
                if actant.name == self.pov_character.name:
                    continue

                user_content_actant = f"""
                Describe the {actant.name} as a novel.
                {actant.name}'s appearance: {actant.appearance}
                """

                response = gpt_response_from_lists(
                        system_content_list = [system_content_first_person_novelist, system_content_POV],
                        user_content_list = [user_content_actant],
                        assistant_content_list = [],
                        model_name = self.model_name
                    ).content

                self.add_to_novel(response)

        # 4, Characterのlogを読み込んで、それを小説形式で書く。
        #    注意点：typeがthoughtであるlogは、thoughtが連続する最初から最後までを一つのthoughtとしてまとめ、要約をする。その要約について、小説化する。
        #    注意点：thought内のneed、goal、world modelといった用語は、要約の際にそれらの用語を用いないようにする。
        # 　　注意点：action_objectについては、それを小説化できるようにプロンプトでサポートする。
        #    注意点：action_objectのconsequenceについては、placeから出ないように制限をしながら、できるだけ小説として敷衍する。
        def is_log_type(log, index, keys, log_type):
            return\
                index <= last_index and\
                type(log[keys[index]]) == dict and\
                log[keys[index]]["type"] == log_type

        thought_list = [] 
        summarized_thought_list = [] 
        self.pov_character.log = self.load_actant_log(self.pov_character.name)
        keys = list(self.pov_character.log.keys())
        last_index = len(self.pov_character.log) - 1
        i = 0

        summarized_thought_list.append(
            f"""
            Information about {self.pov_character.name}:
            - Name: {self.pov_character.name}
            - Personality: {self.pov_character.personality}
            - Appearance: {self.pov_character.appearance}
            - Goal: {self.pov_character.goal}
            - Place: {self.pov_character.place}
            """
        )

        while i < len(keys):
            # <thought>
            is_thought = is_log_type(self.pov_character.log, i, keys, "thought")

            if is_thought:
                thought_list.append(self.pov_character.log[keys[i]]["thought"]["content"])

                while is_log_type(self.pov_character.log, i + 1, keys, "thought"):
                    i += 1
                    thought_list.append(self.pov_character.log[keys[i]]["thought"]["content"])

                # まとめたthoughtsを要約する。
                joined_thought = "\n".join(thought_list)

                system_content_summarize_thought = f"""
                - Name: {self.pov_character.name}
                - Personality: {self.pov_character.personality}
                - Appearance: {self.pov_character.appearance}
                - Place: {self.pov_character.place}

                You are given the thoughts of {self.pov_character.name}. 
                Please summarize the thougt.
                You don't necessarily have to mention Name, Personality, Appearance, Goal, Place in the summary. Only mention them if the content of the thought refers to them.
                The reason you were given knowledge about Name, Personality, Appearance, Goal, Place is to ensure that the summary you make is consistent with the character's information.
                In doing summarization, do not use terms such as need, goal, or world model in your summary.
                These terms have special meanings as follows:
                    Need: A desire that the character temporarily wishes to satisfy. It is also referred to as current need or immediate need.
                    Goal: A long-term objective that the character wishes to achieve.
                    World model: How the character perceives the situation they are in at that scene.
                """

                summarized_thought = gpt_response_from_lists(
                            system_content_list = [system_content_summarize_thought],
                            user_content_list = [joined_thought],
                            assistant_content_list = [],
                            model_name = self.model_name
                        ).content

                thought_list = []

                # summarizeされたthoughtと重複する情報はthoughtから削除する。
                system_content_distill_summarized_thought = f"""
                You are given two pieces of information, thought_history and new_thought.
                Your role is to remove any information from new_thought that is already included in thought_history.
                Please ignore and do not report any information that is in thought_history but not in new_thought.

                Input: thought_history, new_thought
                Output: new_thought that has been removed of any information that is already included in thought_history.
                """

                thought_history = "\n".join(summarized_thought_list)
                new_thought = summarized_thought

                user_content_distill_summarized_thought = f"""
                ```thought_history
                {thought_history}
                ```
                ```new_thought
                {new_thought}
                ```
                """

                distilled_summarized_thought = gpt_response_from_lists(
                            system_content_list = [system_content_distill_summarized_thought],
                            user_content_list = [user_content_distill_summarized_thought],
                            assistant_content_list = [],
                            model_name = self.model_name
                        ).content
                
                # のちほど、思考の内容とactionの描写と整合性を取るために、summarized_thoughtを保存しておく。
                summarized_thought_list.append(distilled_summarized_thought)

                # 要約したthoughtを小説化する。
                self.add_to_novel(
                        gpt_response_from_lists(
                            system_content_list = [system_content_first_person_novelist, system_content_POV],
                            user_content_list = [distilled_summarized_thought],
                            assistant_content_list = [],
                            model_name = self.model_name
                        ).content
                    )

                i += 1
                continue
            # </thought>

            # <action>
            is_action = is_log_type(self.pov_character.log, i, keys, "action")

            if is_action:
                print("is_action")
                system_content_action_note = f"""
                You are given the following information.
                Based on this information, write a part of a novel scene.
                When the name of who_took_action is the same as the name of the POV character, it refers to the POV character themselves.

                - who_took_action: Who took the action
                - action: What they did
                - utterance: What they said when they took the action
                - consequence: What situation arose as a result of the action
                
                When writing a novel based on the consequence, please pay attention to the following points.

                - Interpret the content of the consequence as a summary of the novel. The operation you perform on the consequence is to convert that summary back into a novel.
                - You are allowed to add the actions of the characters to make the content of the consequence natural as a novel.
                - You are allowed to add dialogues of the characters to make the content of the consequence natural as a novel.
                - You are absolutely not allowed to write the continuation of the content of the consequence.
                """
                
                if len(summarized_thought_list) > 0:
                    summarized_thought_text = "\n".join(summarized_thought_list)

                    system_content_action_note += f"""
                    The following is the POV character's thoughts from the start of the scene to now.
                    While you don't need to write this in the main text and it's not recommended to do so, the text you write must be consistent with this.
                    { summarized_thought_text  }
                    """

                action = self.pov_character.log[keys[i]]["action"]

                user_content_action = f"""
                - who_took_action: {action["who_took_action"]}
                - action: {action["action"]}
                - utterance: {action["utterance"]}
                - consequence: {action["consequence"]}
                """

                self.add_to_novel(
                    gpt_response_from_lists(
                            system_content_list = [
                                                    system_content_first_person_novelist,
                                                    system_content_POV,
                                                    system_content_action_note
                                                ],
                            user_content_list = [user_content_action],
                            assistant_content_list = [],
                            model_name = self.model_name
                        ).content
                    )

                i += 1
                continue
            # </action>
            
            # <bare-string>
            is_bare_string = type(self.pov_character.log[keys[i]]) == str

            if is_bare_string:
                self.add_to_novel(
                    gpt_response_from_lists(
                        system_content_list = [system_content_first_person_novelist, system_content_POV],
                        user_content_list = [self.pov_character.log[keys[i]]],
                        assistant_content_list = [],
                        model_name = self.model_name
                    ).content
                )

                i += 1
                continue
            # </bare-string>

            # if none of the above, increment i and continue
            i += 1
            continue
        

        # 5, 4で小説形式で書いたものを、txtファイルとして保存する。
        save_dict_as_json(self.novel, f"{self.pov_character.name}_raw_novel")

        print("[write_raw_novel_first_person] Finished writing raw novel.")

        if styled:
            self.write_styled_novel(style = self.style, language = self.language, pov_name = self.pov_character.name)

# あなたの場面をつくる 
# <a id='createYourScene'>Create Your Scene</a>

場面がどこで行われるかを記述する。<br>
場面はこの場所の中でのみ行われ、他の場所に移ることはない。<br>
describe where the scene takes place.<br>
Scenes take place only in this location, and do not move to any other place.<br>

In [ ]:
place = "A high school classroom after school at 4 p.m. Despite the scattered discarded gym clothes, there is no one around, and the classroom is eerily quiet."

Create Characters
登場人物の作成


| Variable      | Description (EN) | Description (JP) |
| ------------- | ---------------- | ---------------- |
| name | The name of the character. | その人物の名前 |
| personality | The personality of the character and information that only the character can know. | その人物の性格と、その人物だけが知りうる情報 |
| appearance | The appearance of the character and information that other characters also know. | その人物の外見と、他のキャラクターも知っている情報 |
| goal | The goal of the character. The character updates current_need to achieve this goal. | その人物の目標。この目標を満たすように、登場人物はcurrent_needを更新していく |
| current_need | The current need of the character. It is updated to eliminate obstacles to achieving the goal or to directly achieve the goal. This value is updated as the scene progresses. | その人物が現在抱えている欲求。この値は実行中に随時更新される。goalに向かうのに邪魔なものを排除することや、goalに直接向かうことに更新される。 |
| log | A record of the actions the character has taken in the past. This value is updated as the scene progresses. | その人物が過去に行った行動の記録。この値は実行中に随時追加される |

Persolity欄に持っている情報を登場人物同士で秘匿し合うことによってすれ違いを生むことが、この手法での場面生成の根幹である。


In [ ]:
Bocchi = Character(
                name = "Hitori Goto",
                personality = """
                    She perceives herself as a plain, unnoticeable, timid, and cowardly person.
                    Although she is active on the internet under the name "Guitar Hero", she wants to keep it a secret because she believes that if people knew she was the Guitar Hero, it would ruin the image of it.
                    She can tell any lie if it's to hide the fact that she's the Guitar Hero.
                    She always listens to music while commuting to and from school, and she does image training even when she can't touch her guitar.
                    She plays the guitar for at least 4 hours a day, and up to 18 hours at most.
                    She respects Kikuri Hiroi but avoids her because she finds her difficult to deal with.
                    """,
                appearance="""
                    She
                    A 16-year-old high school girl. Since entering high school, she has often been alone.
                    She is considered a weirdo by those around her because she comes to school in a pink tracksuit instead of a uniform and always carries a guitar. However, she doesn't understand why people think she's a weirdo.
                    She speaks in polite language to everyone.
                    """,
                goal = "To keep her identity as the Guitar Hero a secret in real life",
                current_need = "To play the guitar in an empty classroom",
                log = ["'I have to spend another two hours getting home today. I want to become famous and quit high school soon,' said to herself, and  she started playing the guitar in the classroom."],
                place = place
            )



Kikuri = Character(
                name = "Kikuri Hiroi",
                personality = """
                    She's currently drunk, so no matter what you ask her, she'll only respond with 'hehehe, indeed'.
                    She knows that Hitomi Goto is a guitar hero, but she understands well that Hitomi wants to keep it a secret.
                    """,
                appearance= """
                    She/her
                    A 25-year-old woman who works as a bassist.
                    A legendary bassist. She can also play the guitar, but she's better at bass.
                    She has a bad habit of drinking and has caused a lot of trouble, but her fans have always helped her out.
                    There are rumors that she might be the real identity of the guitar hero because of the guitar she owns.
                    She borrowed the guitar from Hitomi Goto that the Guitar Hero uses.
                    """,
                goal = "She lives in the moment, so she doesn't have a goal. If forced to say, her goal is to maximize pleasure in the moment.",
                current_need = "To cause chaos in the situation",
                log = [
                    """
                    Kikuri Hiroi is drunk again today. She realized that she had been borrowing a guitar from her friend, Hitomi Goto, for a while.
                    To return it, she came to Hitori Goto's high school. She trespassed into the classroom and found Hitori Goto there.
                    Seeing Hitori playing the guitar, Kikuri Hiroi forcibly started a guitar session.
                    """]
            )


Hikaru = Character(
                name = "Hikaru Kamiki",
                personality = """
                    He is well aware of his own physical beauty. He believes that his stunning appearance is due to a mission given to him by fate.
                    Furthermore, he feels that he must elevate his own existence to match this mission.
                    He believes that surpassing beings more talented than himself, in other words, killing those individuals, will lead to the elevation of his own existence.
                    Although he has just met Hitori Goto and Kikuri Hiroi, he has immediately recognized their talents.
                    """,
                appearance= """
                    He/Him
                    A man in his 30s with a very beautiful appearance. If you look closely at his eyes, the irises appear like black stars.
                    """,
                goal = "To surpass beings more talented than himself",
                current_need = "To uncover the true identity of a guitar hero active on the internet",
                log = ["""
                    From the guitar hero's Youtube video, Hikaru Kamiki has figured out which regional high school she attends. All that's left is to painstakingly uncover her true identity.
                    Suddenly, he heard the dazzling sound of a guitar. Drawn by the sound, Hikaru Kamiki trespassed into the high school.
                    """],
                place = place
            )



人物ではない場面の構成要素の作成<br>
Create an object that is not a character<br>

| Variable      | Description (EN) | Description (JP) |
| ------------- | ---------------- | ---------------- |
| name | The name of the Item. | そのアイテムの名前 |
| appearance | The appearance of the item.| そのアイテムの見た目 |

In [ ]:
guitar_hitori = Non_Character_Actant(
    name = "Guitar (YAMAHA PACIFICA 611VFM)",
    appearance = "A guitar owned by Hitori Goto. It's different from the guitar used in the Guitar Hero videos.",
    )

actantをリストに格納する。<br>
actantとは、人物と、人物ではない場面の構成要素のことである。<br>
具体的には、CharacterのインスタンスとNon_Character_Actantのインスタンスがactantである。<br>

Store actants in an array. <br>
An actant refers to a character and elements of a scene that are not characters. <br>
Specifically, instances of Character and Non_Character_Actant are actants.

In [ ]:
actants = [
        Bocchi,
        Kikuri,
        Hikaru,
        guitar_hitori,
    ]

場面の最初の出来事を記述する。<br>
Describe the first event of the event.<br>

| Variable      | Description (EN) | Description (JP) |
| ------------- | ---------------- | ---------------- |
| who_took_action | The subject of the action | 誰がその行動をとったか |
| action | The contents of the action | どのような行動が取られたか |
| utterance | What was said when the action was taken | 行動が取られたときの発言 |
| consequence | The result of the action | 行動の結果 |


In [ ]:
catalyst_action_object = Action_Object(
        who_took_action = "Kikuri Hiroi",
        action = """
        Kikuri Hiroi is drunk again today. She realized that she had been borrowing a guitar from her friend, Hitomi Goto, for a while.
        To return it, she came to Hitori Goto's high school. She trespassed into the classroom and found Hitori Goto there.
        Seeing Hitori playing the guitar, Kikuri Hiroi forcibly started a guitar session.
        Kikuri Hiroi finds Kamiki Hikaru wandering the high school corridors as if searching for something. Because he is so handsome, Kikuri invites him into the classroom.
        """,
        utterance = "'Hey, mister over there, come over here!'",
        consequence ="Hitori Goto is flustered because Kamiki Hikaru has entered the classroom. However, Kamiki Hikaru, unfazed, sits on the classroom podium and listens intently to the session between Kikuri Hiroi and Hitori Goto.",
    )



schedule_stackは場面中に発生する状況を記述する。<br>
それらは場面内でスタックの最後の状況から実現される。<br>
スタック内の全ての状況が達成され、1人のキャラクターがそれに対応すると、場面の生成はおわります。<br>
「誰が何をした」ではなく、「誰がどのような状態になった」を記述すること。<br>

The schedule_stack describes the situations that occur in the scene. <br>
They are realized from the last situation in the stack within the scene. <br>
Once all the situations in the stack are achieved, and one character responds to it, the scene generation will finish.<br>
It's not about 'who did what,' but about 'who ended up in what state'.<br>

In [ ]:
schedule_stack = [
    "The fire ignited the alcohol that Hiroi Kikuri had in her mouth. Kamiki Hikaru became a human torch.",
    "Kamiki Hikaru intuitively thought that Hiroi Kikuri is the Guitar Hero. (But he was wrong. Hiroi Kikuri is not the Guitar Hero.)",
]


ゲームマスターが、場面を記述する。<br>
記述された場面は、global_logに格納される。<br>
The game master describes the scene. <br>
The described scene is stored in the `global_log`.<br>

| Variable      | Description (EN) | Description (JP) |
| ------------- | ---------------- | ---------------- |
| actants | Characters and items in the scene | 場面の登場人物とアイテム |
| schedule_stack | Situations that occur in the scene | 場面で起こる状況 |
| tolerance | How many times to allow characters to take actions not in the schedule_stack. This affects the length of the scene. | 登場人物たちに何回までschedule_stack にない行動を許すか。場面の長さに影響を与える。 |
| max_iterations | How many times to iterate the action-response cycle. This affects the length of the scene. | 行動−反応の繰り返しを何回まで行うか。場面の長さに影響を与える。 |


In [ ]:
game_master = Game_Master(
                actants=actants,
                schedule_stack=schedule_stack,
                tolerance= 1,
            )

game_master.main(catalyst_action_object=catalyst_action_object, max_iterations=20)

ここまでで、小説として描かれるべき場面の出来事が生成される。その内容を、`result/`配下に保存する。
Events suitable for novel scenes are generated. Store them in the `result/` directory.

In [ ]:
global_str = json.dumps(global_log.log, indent = 4)
with open('result/global_log.txt', 'w') as f:
    f.write(global_str)


# for each actant, if its type is Character, then save their log
for actant in game_master.actants:
    if type(actant) == Character:
        actant_str = json.dumps(actant.log, indent = 4)
        with open(f'result/{actant.name}_log.txt', 'w') as f:
            f.write(actant_str)
    

一人称視点の小説を描く。
Write a first-person novel.

文体の指定。以下のように細々と指示をすることもできるし、「ライトノベル調で書いてください」のような短い指示でもよい。

In [ ]:
style_hard = f"""
- 文体は三人称小説でありつつ、視点は{Bocchi.name}の一人称であること。
  例；「私はりんごを食べた。」ではなく、「{Bocchi.name}はりんごを食べた。」と書くこと。
- 重文・複文は用いないこと。重文・複文を用いる代わりに、句点で文を区切ったうえで接続詞を用いた文を続けること。
- 同一の段落の2つ目以降の文は、既出の単語または接続詞と既出の単語から始めること。
- 比喩は用いず、簡潔な表現を用いること。
- 「彼」や「彼女」などの代名詞は用いず、名詞を用いること。
- 筆者による価値判断は行わないこと。登場人物による動作でそれを示唆すること。
    <examples>
        <example class="筆者による価値判断">赤い美しい花が咲いていた。</example>
        <example class="登場人物の動作による示唆">赤い花が咲いていた。{{登場人物}}は足を止め、しばらくその花に見入った。</example>
    </examples>
- 文体は敬体でなく常体で書くこと。また、終戦直後の日本語の文体に近づけること。
    <examples>
        <example class="常体">日本国は憲法を持つ。</example>
        <example class="敬体">日本国は憲法を持ちます。</example>
    </examples>
- 文は必ず主語か主題、またはその両方を含むこと。主語には、格助詞の「は」か「が」がつく。主題には格助詞の「は」がつく。
- 「だろう」や「思う」のような言葉を使って断定を避けてはならない。「だ」、「である」を用いて断言をすること。
   やむをえず断言をすることができない場合は、「〜である可能性は高い。」や「〜と判断する根拠は乏しい」のような言葉を使い、「だろう」や「思う」を使うことは避けてください。
"""

style_light = f"""
- 文体は一人称小説でありつつ、視点は{Bocchi.name}の一人称であること。
- 16歳の少女の文体で書くこと。すなわち、あらゆることを彼女の興味の分野であるロックやギターやお菓子に例えること。
- 「彼」や「彼女」などの代名詞は用いず、名詞を用いること。
- 地の文の登場人物の名前はすべて、カタカナで書くこと。
- 各登場人物は、以下のように互いを呼ぶ。
    Goto Hitori calls Kamiki Hikaru "カミキさん"
    Goto Hitori calls Hiroi Kikuri "おねえさん"
    Goto Hitori calls himself "わたし"
    Kamiki Hikaru calls Goto Hitori "ゴトウさん"
    Kamiki Hikaru calls Hiroi Kikuri "ギターのうまいお姉さん"
    Kamiki Hikaru calls himself "ぼく"
    Hiroi Kikuri calls Goto Hitori "ぼっちちゃん"
    Hiroi Kikuri calls Kamiki Hikaru "イケメン"
    Hiroi Kikuri calls herself "あたし"
"""

小説を書くためのクラスであるNovelistのインスタンスをつくる。<br>
Create an instance of the Novelist class for writing a novel.<br>

<table>
    <thead>
        <tr>
            <th>Variable Name</th>
            <th>Description (En)</th>
            <th>Description (Jp)</th>
        </tr>
    <thead>
    <tbody>
        <tr>
            <td>pov_character</td>
            <td>The Character instance that serves as the first-person perspective in the novel.</td>
            <td>小説の一人称視点となる人物のCharacterインスタンス。</td>
        </tr>
        <tr>
            <td>initial_actants</td>
            <td>The initial Character instances that interact with the pov_character in the novel.</td>
            <td>小説の一人称視点となる人物のCharacterインスタンス。</td>
        </tr>
        <tr>
            <td>language</td>
            <td>The language in which the novel is written.</td>
            <td>小説を何語で書くか</td>
        </tr>
        <tr>
            <td>style</td>
            <td>The style in which the novel is written.</td>
            <td>小説をどのような文体で書くか</td>
        </tr>
    </tbody>
</table>

pov_characterのnameに従って、`result/`配下から、対応するlogが読み込まれ、それを小説化していく。<br>
小説として描写する順序は、以下の通りである。<br>
- Characterのplace<br>
- Character自身<br>
- Character自身以外のactantsのappearance<br>
<br>
According to the name of pov_character, the corresponding log is loaded from `result/`, and it is novelized. The order of description as a novel is as follows:<br>
- The place of the Character<br>
- The Character itself<br>
- The appearance of actants other than the Character itself<br>


In [ ]:
Bocchi_novelist = Novelist(
        pov_character = Bocchi,
        initial_actants = actants,
        language = "Japanese",
        style = style_light,
        model_name = "gpt-4",
    )

小説を執筆する。
Write Novel.

In [ ]:
Bocchi_novelist.write_raw_novel_first_person(styled = False)

In [ ]:
Bocchi_novelist.write_styled_novel()